## Zadanie domowe: Projektowanie filtru metodą okien

Zauważone w zadaniu z filtracją artefakty są efektem zastosowania filtru idealnego (wycięcia odpowiednich częstotliwości).
Aby je usunąć, należy filtr idealny w dziedzinie przestrzennej przemnożyć przez dwuwymiarowe okno (stworzone na podstawie jednowymiarowego).

1. Zdefiniuj rozmiar filtra (np. 21).

2. Stwórz okno np. Hanninga (`np.hanning`) o zadanym rozmiarze.

3. Na podstawie tego okna stwórz okno dwuwymiarowe: $F = f^Tf$.
Można to zrealizować prostą instrukcją:

        hanning2d = np.outer(hanning, hanning)

4. Stwórz wzorcowy filtr idealny tak samo jak w poprzednim ćwiczeniu.
Jego rozmiar musi być taki sam jak zdefiniowany w punkcie pierwszym.

5. Generalna zasada jest taka, że należy obliczyć odwrotną transformatę Fouriera filtru idealnego.
Aby zagwarantować jednak dobrą lokalizację i symetryczność tworzonego filtru wykorzystuje się dodatkowe rotacje i przesunięcia.

        FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
        FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
        FilterFidft = cv2.idft(np.float32(FilterFRot3),
                        flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
        FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

6. Aby otrzymać poprawny filtr należy teraz pomnożyć dwuwymiarowe okno ze stworzonym filtrem idealnym w dziedzinie przestrzennej.

7. Stworzony filtr powinien zostać znowu przeniesiony do dziedziny częstotliwości.
Należy jednak rozszerzyć go tak, aby miał taki sam rozmiar jak filtrowany obraz.
Operację tę wykonaj tak samo jak w poprzednim ćwiczeniu (`cv2.copyMakeBorder`).
Oblicz transformatę Fouriera i wykonaj przesunięcie.
Następnie oblicz wartość bezwzględną (dzięki temu filtr nie będzie wpływał na fazę).
Wynikiem jest poprawiony filtr w dziedzinie częstotliwości.

8. Przeprowadź filtrację obrazu *lena.bmp* za pomocą zaprojektowanego filtra.
Wyświetl filtr, oraz wynik filtracji.
Do wyświetlenia filtru możesz użyć przestrzeń częstotliwości wygenerowaną w poprzednim zadaniu (*FSpaceRowsM* i *FSpaceColsM*).

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

# Load required files
if not os.path.exists("lena.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/lena.bmp --no-check-certificate

I_lena = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)      

In [ ]:
# funkcje z zadania podstawowego

def fourier(I):
    I = I.astype("float32")
    fcn = cv2.dft(I,flags=cv2.DFT_COMPLEX_OUTPUT)
    fcn = np.fft.fftshift(fcn, [0,1])
    magnitude, phase = cv2.cartToPolar(fcn[:,:,0], fcn[:,:,1])
    magnitude_log = np.log10(magnitude + 1)
    return fcn, magnitude_log, phase

def invertion(F_img):
    I_s = np.fft.ifftshift(F_img, [0,1])
    ifft = cv2.idft(I_s, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    I_ifft = cv2.magnitude(ifft[:, :, 0], ifft[:, :, 1])
    I_ifft = np.round(I_ifft).astype('uint8')
    return I_ifft


In [ ]:
window_size=[21, 21]
hanning = np.hanning(21)
hanning2d = np.outer(hanning, hanning)

FSpaceRows = 2*np.fft.fftshift(np.fft.fftfreq(window_size[0]))
FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, window_size[1]]))
FSpaceCols = 2*np.fft.fftshift(np.fft.fftfreq(window_size[1]))
FSpaceColsM = np.outer(np.ones([1, window_size[0]]), FSpaceCols)    
FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))

FilterF = FreqR >= 0.2

FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
FilterFidft = cv2.idft(np.float32(FilterFRot3),flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

Filtr = hanning2d * FilterFI

Filtr_border = cv2.copyMakeBorder(Filtr, 246, 245, 246, 245, cv2.BORDER_CONSTANT,value=0)

Filtr_border = Filtr_border.astype("float32")
fcn_filtr = cv2.dft(Filtr_border,flags=cv2.DFT_COMPLEX_OUTPUT)
fcn_filtr = np.fft.fftshift(fcn_filtr, [0,1])

fcn_filtr_abs = np.abs(fcn_filtr)

lenaSize=I_lena.shape
FSpaceRows512 = 2*np.fft.fftshift(np.fft.fftfreq(lenaSize[0]))
FSpaceRowsM512 = np.outer(FSpaceRows512, np.ones([1, lenaSize[1]]))
FSpaceCols512 = 2*np.fft.fftshift(np.fft.fftfreq(lenaSize[1]))
FSpaceColsM512 = np.outer(np.ones([1, lenaSize[0]]), FSpaceCols512)  


figFilter = plt.figure()
axsFilter = figFilter.add_subplot(projection='3d')
axsFilter.plot_surface(FSpaceRowsM512, FSpaceColsM512, fcn_filtr_abs[:,:,0], rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)

fcn_lena, _, _ = fourier(I_lena)

lena_filtr = fcn_lena * fcn_filtr_abs
lena_ifft = invertion(lena_filtr)

f, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(15,10))
ax1.imshow(I_lena, 'gray')
ax1.set_title("Obraz oryginalny")
ax1.axis('off')
ax2.imshow(lena_ifft, 'gray')
ax2.set_title("Odwrotna transformata")
ax2.axis('off')
ax3.imshow(np.abs(I_lena - lena_ifft), 'gray')
ax3.set_title("Różnica")
ax3.axis('off')
plt.show()
